#  Домашнее задание №2
## Алиев М.А. группа 154
### Тема - извлечение устойчивых словосочетаний

In [81]:
import nltk
import math
# hand-made ;)
tok = Tokenizer()

# прочитаем текст из файла
with open('Гоголь.txt', 'r') as f:
    text = f.read()
    
# я использую токенизатор, созданный мной в 1ом домашнем задании.
tokens = tok.tokenize(text)

print (len(tokens))

117655


In [73]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

# лемматизируем все токены
# это может занять некоторове время! (около 30 сек. на Ubuntu 16.04)
lemmas = [morph.parse(token)[0].normal_form for token in tokens]

In [79]:
from nltk.corpus import stopwords

nltk.download('stopwords')
# избавимся от стоп слов

tokens_c = [token for token in lemmas if token not in set(stopwords.words('russian'))]

[nltk_data] Downloading package stopwords to /home/maga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [82]:
from collections import Counter

c = Counter(tokens_c)
# посчитаем 10 самых частых в употреблении слов
print(c.most_common(10))

[('весь', 1273), ('это', 845), ('сказать', 834), ('чичиков', 780), ('который', 688), ('человек', 503), ('свой', 444), ('мочь', 411), ('дело', 406), ('говорить', 392)]


In [222]:
from collections import defaultdict

freq_word = 'дело'
# найдем все биграммы, содержащие выбранное нами слово
bigrams = []
real_bigrams = defaultdict(lambda: [])

for (num, word) in enumerate(lemmas):
    if word != freq_word: continue
    # проверяем индекс, чтобы не выйти за границы массива
    if num > 0:
        # будем также сохранять и примеры из текста без лемматизации (разные формы нашей биграммы)
        bigrams.append((lemmas[num-1], word))
        real_bigrams[(lemmas[num-1], word)] += [(tokens[num-1].lower(), tokens[num].lower())]
    if num < len(lemmas) - 1:
        bigrams.append((word, lemmas[num+1]))
        real_bigrams[(word, lemmas[num+1])] += [(tokens[num].lower(), tokens[num+1].lower())]

In [230]:
# также созадим и список со всеми возможными биграммами текста
all_bigrams = [(lemmas[i], lemmas[i + 1]) for i in range(len(lemmas) - 1)]
all_bigrams_real = defaultdict(lambda: [])
for i in range(len(lemmas) - 1):
    # опять же, для каждой лемматизированной биграммы, будем сохранять ее формы
    all_bigrams_real[(lemmas[i], lemmas[i + 1])] += [(tokens[i].lower(), tokens[i+1].lower())]

In [244]:
# Посчитаем частоту биграмм с нужным нам словом обычным счетом (всего биграмм со словом/данная биграмма)
counter = Counter(bigrams)
for bi in counter.most_common(20):
    real_ = [i[0] for i in Counter(real_bigrams[bi[0]]).most_common(2)]
    if (len(real_) < 2): real_.append(('', ''))
    print ("'{:7} {:6}' -> кол-во: {:3}, отн. частота: {:.5f}, примеры: '{} {}', '{} {}'".format(
                bi[0][0],
                bi[0][1],
                bi[1],
                bi[1]/len(all_bigrams),
                real_[0][0], real_[0][1],
                real_[1][0], real_[1][1]
            )
        )

'самый   дело  ' -> кол-во:  59, отн. частота: 0.00050, примеры: 'самом деле', 'самого дела'
'дело    и     ' -> кол-во:  21, отн. частота: 0.00018, примеры: 'дело и', 'дела и'
'это     дело  ' -> кол-во:  20, отн. частота: 0.00017, примеры: 'это дело', 'этом деле'
'дело    в     ' -> кол-во:  18, отн. частота: 0.00015, примеры: 'дело в', 'деле в'
'и       дело  ' -> кол-во:  17, отн. частота: 0.00014, примеры: 'и дело', 'и дела'
'дело    что   ' -> кол-во:  12, отн. частота: 0.00010, примеры: 'дело что', 'деле что'
'дело    быть  ' -> кол-во:  12, отн. частота: 0.00010, примеры: 'дело было', 'дело будет'
'дело    не    ' -> кол-во:  12, отн. частота: 0.00010, примеры: 'дело не', 'деле не'
'в       дело  ' -> кол-во:  12, отн. частота: 0.00010, примеры: 'в дело', 'в деле'
'другой  дело  ' -> кол-во:  10, отн. частота: 0.00008, примеры: 'другое дело', 'других дел'
'главный дело  ' -> кол-во:   9, отн. частота: 0.00008, примеры: 'главное дело', ' '
'дело    а     ' -> кол-во:   9, отн. ч

In [262]:
# теперь посчитаем по мере Dice (для этих же биграм)
counter_words = Counter(lemmas)

dice_metric_word = [(0, (0, 0))] * len(set(bigrams))
for (num, bi) in enumerate(set(bigrams)):
    dice_metric_word[num] = (2*counter[bi]/(counter_words[bi[0]] + counter_words[bi[1]]), bi)
    
for bi in sorted(dice_metric_word, reverse=True, key=lambda x: x[0])[:20]:
    real_ = [i[0] for i in Counter(real_bigrams[bi[1]]).most_common(2)]
    if (len(real_) < 2): real_.append(('', ''))
    print ("'{:10} {:5}' -> мера: {:.5f}, кол-во: {:3}, примеры: '{} {}', '{} {}'".format(
            bi[1][0],
            bi[1][1],
            bi[0], counter[bi[1]],
            real_[0][0], real_[0][1],
            real_[1][0], real_[1][1]
        )
    )

'самый      дело ' -> мера: 0.16643, кол-во:  59, примеры: 'самом деле', 'самого дела'
'главный    дело ' -> мера: 0.04196, кол-во:   9, примеры: 'главное дело', ' '
'это        дело ' -> мера: 0.03197, кол-во:  20, примеры: 'это дело', 'этом деле'
'другой     дело ' -> мера: 0.02538, кол-во:  10, примеры: 'другое дело', 'других дел'
'дело       идти ' -> мера: 0.02537, кол-во:   6, примеры: 'дело идет', 'дело идут'
'важный     дело ' -> мера: 0.02381, кол-во:   5, примеры: 'важным делом', 'важные дела'
'чем        дело ' -> мера: 0.02335, кол-во:   6, примеры: 'чем дело', ' '
'бесчестный дело ' -> мера: 0.01923, кол-во:   4, примеры: 'бесчестнейшее дело', 'бесчестных дел'
'дело       но   ' -> мера: 0.01731, кол-во:   9, примеры: 'дело но', 'дел но'
'ваш        дело ' -> мера: 0.01709, кол-во:   5, примеры: 'ваше дело', 'вашему делу'
'за         дело ' -> мера: 0.01540, кол-во:   7, примеры: 'за дело', 'за делом'
'дело       по   ' -> мера: 0.01466, кол-во:   7, примеры: 'дела по', 'д

In [265]:
# посчитаем по MI (снова для наших биграм)
N = len(Counter(lemmas))
mi_metric_word =  [(0, (0, 0))] * len(set(bigrams))
for (num, bi) in enumerate(set(bigrams)):
    mi_metric_word[num] = (math.log(N * counter[bi]/(counter_words[bi[0]] * counter_words[bi[1]]), 2), bi)    

for bi in sorted(mi_metric_word, reverse=True, key=lambda x: x[0])[:20]:
    real_ = [i[0] for i in Counter(real_bigrams[bi[1]]).most_common(2)]
    if (len(real_) < 2): real_.append(('', ''))
    print ("'{:14} {:8}' -> мера: {}, кол-во: {:3}. примеры: '{} {}', '{} {}'".format(
            bi[1][0],
            bi[1][1],
            bi[0], counter[bi[1]],
            real_[0][0], real_[0][1],
            real_[1][0], real_[1][1]
        )
    )

'отлагать       дело    ' -> мера: 4.901907188094263, кол-во:   1. примеры: 'отлагая дела', ' '
'христолюбивый  дело    ' -> мера: 4.901907188094263, кол-во:   1. примеры: 'христолюбивое дело', ' '
'распутывать    дело    ' -> мера: 4.901907188094263, кол-во:   1. примеры: 'распутывать дело', ' '
'премерзкий     дело    ' -> мера: 4.901907188094263, кол-во:   1. примеры: 'премерзейшее дело', ' '
'дело           беспримерный' -> мера: 4.901907188094263, кол-во:   1. примеры: 'деле беспримерное', ' '
'обделывать     дело    ' -> мера: 4.901907188094263, кол-во:   1. примеры: 'обделывать дела', ' '
'запутаннейший  дело    ' -> мера: 4.901907188094263, кол-во:   2. примеры: 'запутаннейшее дело', 'запутаннейшего дела'
'дело           неисправимый' -> мера: 4.901907188094263, кол-во:   1. примеры: 'дела неисправимого', ' '
'небесчестный   дело    ' -> мера: 4.901907188094263, кол-во:   1. примеры: 'небесчестного дела', ' '
'головоломный   дело    ' -> мера: 4.901907188094263, кол-во:   1. пр

In [266]:
# Теперь найдем по 20 самых устойчивых биграм (по всему тексту) с помощью обеих метрик
# Сначала по Дайсу
dice_metric = [0] * len(set(all_bigrams))
for (num, bi) in enumerate(set(all_bigrams)):
    dice_metric[num] = (2*counter_all[bi]/(counter_words[bi[0]] + counter_words[bi[1]]), bi)
    
for bi in sorted(dice_metric, reverse=True, key=lambda x: x[0])[:20]:
    real_ = [i[0] for i in Counter(all_bigrams_real[bi[1]]).most_common(2)]
    if (len(real_) < 2): real_.append(('', ''))
    print ("'{:14} {:12}' -> мера: {:.5f}, кол-во: {:3}, примеры: '{} {}'".format(
            bi[1][0],
            bi[1][1],
            bi[0], counter_all[bi[1]],
            real_[0][0], real_[0][1],
            real_[1][0]
        )
    )

'укокошить      неантихрист ' -> мера: 1.00000, кол-во:   1, примеры: 'укокошили неантихристов'
'еше            куриться    ' -> мера: 1.00000, кол-во:   1, примеры: 'еше курилась'
'полмиллиона    сидней      ' -> мера: 1.00000, кол-во:   1, примеры: 'полмиллиона сидней'
'штаб-офицер    брандер     ' -> мера: 1.00000, кол-во:   1, примеры: 'штаб-офицеров брандеров'
'двухаршинный   стерлядь    ' -> мера: 1.00000, кол-во:   1, примеры: 'двухаршинными стерлядями'
'покрытие       бренный     ' -> мера: 1.00000, кол-во:   1, примеры: 'покрытия бренного'
'федосея        федосей     ' -> мера: 1.00000, кол-во:   1, примеры: 'федосей федосеевич'
'понукание      нацеплять   ' -> мера: 1.00000, кол-во:   1, примеры: 'понуканьях нацепляя'
'незапамятный   младенчество' -> мера: 1.00000, кол-во:   1, примеры: 'незапамятного младенчества'
'утончённейший  гастроном   ' -> мера: 1.00000, кол-во:   1, примеры: 'утонченнейшего гастронома'
'сомовий        плёс        ' -> мера: 1.00000, кол-во:   3, прим

In [268]:
# Теперь по MI
mi_metric = [0] * len(set(all_bigrams))
for (num, bi) in enumerate(set(all_bigrams)):
    mi_metric[num] = (math.log(N*counter_all[bi]/(counter_words[bi[0]] * counter_words[bi[1]]), 2), bi)
    
for bi in sorted(mi_metric, reverse=True, key=lambda x: x[0])[:20]:
    real_ = [i[0] for i in Counter(all_bigrams_real[bi[1]]).most_common(2)]
    if (len(real_) < 2): real_.append(('', ''))
    print ("'{:14} {:12}' -> мера: {:.5f}, кол-во: {:3}, примеры: '{} {}'".format(
            bi[1][0],
            bi[1][1],
            bi[0], counter_all[bi[1]],
            real_[0][0], real_[0][1],
            real_[1][0]
        )
    )

'укокошить      неантихрист ' -> мера: 13.56724, кол-во:   1, примеры: 'укокошили неантихристов'
'еше            куриться    ' -> мера: 13.56724, кол-во:   1, примеры: 'еше курилась'
'полмиллиона    сидней      ' -> мера: 13.56724, кол-во:   1, примеры: 'полмиллиона сидней'
'штаб-офицер    брандер     ' -> мера: 13.56724, кол-во:   1, примеры: 'штаб-офицеров брандеров'
'двухаршинный   стерлядь    ' -> мера: 13.56724, кол-во:   1, примеры: 'двухаршинными стерлядями'
'покрытие       бренный     ' -> мера: 13.56724, кол-во:   1, примеры: 'покрытия бренного'
'федосея        федосей     ' -> мера: 13.56724, кол-во:   1, примеры: 'федосей федосеевич'
'понукание      нацеплять   ' -> мера: 13.56724, кол-во:   1, примеры: 'понуканьях нацепляя'
'незапамятный   младенчество' -> мера: 13.56724, кол-во:   1, примеры: 'незапамятного младенчества'
'утончённейший  гастроном   ' -> мера: 13.56724, кол-во:   1, примеры: 'утонченнейшего гастронома'
'пикендрас      пичурущуха  ' -> мера: 13.56724, кол-во

Текстовое описание и интерпритация полученных результатов находится в отдельном файле отчета.